In [ ]:
import gc
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
# import seaborn as sns
# from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import futuresalesutility as fu

In [ ]:
m1 = pd.read_csv('valid_my_stuff.csv.gz')
m2 = pd.read_csv('valid_extra_features.csv.gz')
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
m1 = m1.merge(items[['item_id', 'item_name']], on='item_id', how='left')
m2 = m2.merge(items[['item_id', 'item_name']], on='item_id', how='left')

In [ ]:
train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
train["date"] = pd.to_datetime(train["date"], format="%d.%m.%Y")

In [ ]:
m1['err'] = np.abs(m1.item_cnt_month - m1.preds.clip(0,20))
m2['err'] = np.abs(m2.item_cnt_month - m2.preds.clip(0,20))
m1['errdiff'] = m1['err'] - m2['err']

In [ ]:
grouper = ['new_item']
basic = pd.DataFrame(m1.groupby(grouper).err.mean().rename('basic'))
extra = m2.groupby(grouper).err.mean().rename('extra')

basic = basic.merge(extra, left_index=True, right_index=True)
basic = basic.sort_values('basic', ascending=True)
ax = basic.plot.barh(figsize=(10,10))
# _ = plt.title('General categories in train and test sets')
# _ = plt.xlabel('proportion')

In [ ]:
m1[['item_id', 'item_name_group', 'item_name', 'item_category_id', 'item_cnt_month', 'preds', 'err', 'errdiff']].nlargest(50, 'errdiff')

In [ ]:
m2[['item_id', 'item_name', 'item_category_id', 'item_cnt_month', 'preds', 'err']].nlargest(30, 'err')

In [ ]:
import shap
temp = X_test.drop(columns='lgb_pred').sample(10000)
explainer = shap.TreeExplainer(lgb_model)
shap_values = explainer.shap_values(temp)
shap.summary_plot(shap_values, temp, max_display=30)